In [1]:
import numpy as np
import pandas as pd
import h5py
import scipy
import scanpy as sc
import anndata
from scipy.sparse import find
from scbasset.utils import *
import seaborn as sns
import matplotlib.pyplot as plt
import os

/tmp/ipykernel_8561/2271081770.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-16 09:06:55.323070: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-16 09:06:55.872592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 09:06:55.872695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin 

# Choices

In [ ]:
#########################
# selection of the data #
#########################

type_data = 'enhancer'
# type_data = 'promotor'
# type_data = 'enhancer_promotor'
# type_data = 'enhancer_SCAFE'



###############
# Filter Info #
###############

#filter_info = 'filtered'
#filter_info = 'not_filtered'
filter_info = 'filtered_controle'

#peak need to be accessible in _ % cells
filter_rate = 0.05

#path of the count matrix and peaks.bed
data_path = f'../../../results_scbasset/{type_data}/data/'



#########
# Input #
#########

count_matrix_file = f'{data_path}count_matrix_{type_data}.h5'
bed_file = f'{data_path}peaks_{type_data}.bed'



##########
# Output #
##########

output_anndata = f'{data_path}{filter_info}/cage_{type_data}_ad.h5ad'


# Read the count_matrix and peaks

In [5]:
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end', 'strand'])
ad = sc.read_10x_h5(count_matrix_file, gex_only=False)

In [6]:
print(f'Number of {type_data}: {ad.shape[1]} \nNumber of patient: {ad.shape[0]}')

Number of promotor: 24698 
Number of patient: 98


# Filtering

## Filtering on peak accessibilities in patient 

In [10]:
ad_cage = ad[:, ad.var['feature_types']=='Peaks']
ad_cage.var['chr'] = peak['chr'].values
ad_cage.var['start'] = peak['start'].values
ad_cage.var['end'] = peak['end'].values
ad_cage.var['strand'] = peak['strand'].values

# # basic stats
sc.pp.filter_cells(ad_cage, min_genes=0)
sc.pp.filter_genes(ad_cage, min_cells=0)

# # a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*filter_rate)
ad_cage = ad_cage[:, ad_cage.var['n_cells']>=thres]

/tmp/ipykernel_8561/2389924254.py:2: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  ad_cage.var['chr'] = peak['chr'].values


In [12]:
print(f'Number of {type_data}: {ad_cage.shape[1]} \nNumber of patient: {ad_cage.shape[0]}')

Number of promotor: 24698 
Number of patient: 98


## Filtering on chromosomes

In [13]:
chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
ad_cage = ad_cage[:, ad_cage.var['chr'].isin(chrs)]

In [14]:
print(f'Number of {type_data}: {ad_cage.shape[1]} \nNumber of patient: {ad_cage.shape[0]}')

Number of promotor: 24644 
Number of patient: 98


### save h5ad

In [15]:
ad_cage.write(output_anndata)

/home/pascal/Norway/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/pascal/Norway/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/pascal/Norway/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
/home/pascal/Norway/lib/python3.10/site-packages/anndata/_core/anndata.py:1301: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c
